In [ ]:
import huggingface_hub
from stable_diffusion_reference_only.pipelines.stable_diffusion_reference_only_pipeline import (
    StableDiffusionReferenceOnlyPipeline,
)
import anime_segmentation
from diffusers.schedulers import UniPCMultistepScheduler
from PIL import Image
import cv2
import numpy as np
import os
import torch

In [ ]:
device = "cuda"
dtype = torch.float16

In [ ]:
automatic_coloring_pipeline = StableDiffusionReferenceOnlyPipeline.from_pretrained(
    "AisingioroHao0/stable-diffusion-reference-only-automatic-coloring-0.1.2"
).to(device, dtype=dtype)
automatic_coloring_pipeline.scheduler = UniPCMultistepScheduler.from_config(
    automatic_coloring_pipeline.scheduler.config
)
segment_model = anime_segmentation.get_model(
    model_path=huggingface_hub.hf_hub_download("skytnt/anime-seg", "isnetis.ckpt")
).to(device, dtype=dtype)

In [ ]:
prompt = (
    Image.open("../validation_images/3/1_capture.jpg").convert("RGB").resize((512, 512))
)
prompt

In [ ]:
blueprint = Image.open("../validation_images/9/2_line.jpg").convert("RGB").resize((512, 512))
blueprint = Image.eval(blueprint, lambda x: 255 - x)
blueprint

In [ ]:
result = automatic_coloring_pipeline(
    prompt=prompt, blueprint=blueprint, num_inference_steps=20
).images[0]
result

In [ ]:
new_result=Image.new("RGB",(512*3,512))
new_result.paste(prompt,(0,0))
new_result.paste(blueprint,(512,0))
new_result.paste(result,(1024,0))
new_result.save(os.path.join("../lab_logs","12.jpg"))
new_result